# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [1]:
# verifying pathlib is clean
import pathlib
print(pathlib.__file__)

/usr/lib/python3.10/pathlib.py


In [2]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import time
import torch
import numpy as np

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxend module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = True
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

/usr/lib/python3.10/pathlib.py
***ExpRun**: Loading pointer config file:
	/home/al5d/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	~/WORK/BerryPicker/cfg/settings.yaml
Using device: cuda


In [3]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

Hostname is raven
Path for external experiments: /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun
Path for external data: /home/al5d/WORK/_DataExternal/BerryPicker-BC/result
***ExpRun**: Experiment config path changed to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun
***ExpRun**: Experiment data path changed to /home/al5d/WORK/_DataExternal/BerryPicker-BC/result
***ExpRun**: Experiment demonstration copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/robot_al5d
***ExpRun**: Experiment automate copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/automate
***ExpRun**: Experiment behavior_cloning copied to /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/behavior_cloning
***ExpRun**: Experiment controllers copied 

In [4]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
run = "runbc_bc_lstm_0001"
exp = Config().get_experiment(experiment, run)

exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])
exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

['bc_run_0001',
 'bc_verify_mlp_00',
 'runbc_bc_lstm_0001',
 'bc_verify_lstm_mdn_00',
 'mdn_00',
 'bc_mlp_00',
 'bc_lstm_00',
 'bc_lstm_mdn_00',
 'runbc_bc_mlp_0001',
 'bc_mlp_0001',
 'bc_lstm_resid_00',
 'mdn_for_bc_00',
 'bc_lstm_0001',
 'bc_verify_lstm_resid_00',
 'bc_lstm_residual_0001',
 'bc_lstm_mdn_0001',
 'bc_compare_00',
 '_defaults_behavior_cloning',
 'runbc_bc_lstm_residual_0001',
 'bc_0001',
 'bc_run_00',
 'runbc_bc_lstm_mdn_0001',
 'bc_verify_lstm_00']
***ExpRun**: Configuration for exp/run: behavior_cloning/runbc_bc_lstm_0001 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/_DataExternal/BerryPicker-BC/exprun/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_cam0_controller successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_0001 successfully loaded
***ExpRun*

In [5]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path))   

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded


***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
cap2 works


<All keys matched successfully>

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

FIXME: this controller loop needs to be made into a function and cleaned up. 

In [ ]:
last_interval = 0
controller_interval = 0.1

transform = sp_helper.get_transform_to_sp(exp_sp)
# the backlog of the last sequence length steps
z_seq = []

while True:
    start_time = time.time() 
    # Let us assume that the first image here is the one
    camera_controller.update()
    image = camera_controller.images[exp["control_camera"]]
    # FIXME: this is totally won't work like this
    # there should be a notebook with the input processing???
    sensor_readings = sp_helper.load_capture_to_tensor(image, transform, device)
    z = sp.process(sensor_readings[0])
    print(z)
    if exp_bc["sequence_length"]: # no sequence processsing
        z_seq.append(z)
        if len(z_seq) > exp_bc["sequence_length"]:
            z_seq.pop(0)
        if len(z_seq) < exp_bc["sequence_length"]:
            ## we don't have enough in the queue, wait.
            print("Not acting, waiting on the queue", flush=True)
            z_tensor = None
            pass
        else:
            # FIXME: this should be applied online, not step by step
            z_seq_np = np.array(z_seq)
            z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(device)
    else:
        z_tensor = torch.tensor(z).unsqueeze(0).to(device)
    if z_tensor == None:
        continue
    if not model.stochastic:                
        a_pred = model(z_tensor)
    else:
        a_pred = model.forward_and_sample(z_tensor)

    print(a_pred, flush=True)

    if robot_controller:
        # move the robot
        pos_target = RobotPosition.from_normalized_vector(exp_robot_controller,a_pred[0])

        # ask the user whether we indeed want to continue moving?
        pos_current = robot_controller.get_position()
        distance = pos_current.empirical_distance(exp_robot_controller, pos_target)
        print(f"Next position: {pos_target} at distance={distance}")
        ans = input("Proceed with move? (Y/N/Q)")
        if ans.lower() == "y":
            robot_controller.move(pos_target)
        elif ans.lower() == "q":
            break # break out from the loop
    else:
        pass

    # ensuring that the execution happens 
    end_time = time.time() 
    execution_time = end_time - start_time 
    last_interval = execution_time
    time_to_sleep = max(0.0, controller_interval - execution_time) 
    time.sleep(time_to_sleep) 


[ 1.1389407e+00  3.0960077e-01 -3.4976330e-01  1.3536881e-01
  6.6381717e-01 -3.1074697e-01 -4.8035547e-01 -5.3151798e-01
 -2.6609400e-01  7.2726968e-04  3.2855543e-01  1.4531620e-01
  3.7600055e-01  9.9270636e-01 -1.7153831e-01 -3.7856019e-01
  1.4210664e-01 -6.0119140e-01  4.4153467e-01 -4.0350029e-01
  8.4908016e-02  1.8527141e-01 -6.9388561e-03  6.9223630e-01
 -4.9244300e-01  3.7885028e-01 -1.3349529e-01 -5.2381611e-01
 -3.8252270e-01  6.5736912e-02  3.2141036e-01  6.0343480e-01
 -3.7355590e-01 -3.5243121e-01  1.5673599e-01 -3.7109753e-01
 -2.7642760e-01  1.4004280e-01  4.3842486e-01 -5.6571734e-01
  2.1281213e-01 -4.5770642e-01  4.2315599e-01  3.8355088e-01
  9.6038133e-02 -7.0682490e-01 -2.0028499e-01  2.5856841e-01
  3.7536561e-01  3.3594154e-02 -2.4727909e-01 -2.3756304e-01
 -8.4519446e-01 -1.4780864e-01  2.8061146e-01  6.7319340e-01
  6.3079424e-02  5.7438320e-01 -5.6926113e-01  6.8257666e-01
 -7.7841304e-02  5.3805351e-01  2.6918656e-01 -2.6836461e-01
  4.2986780e-01  5.03191

[ 1.14010251e+00  3.09963882e-01 -3.48741859e-01  1.34426713e-01
  6.63357675e-01 -3.09808075e-01 -4.80044365e-01 -5.31406522e-01
 -2.65590638e-01  1.71300385e-03  3.29638571e-01  1.46109372e-01
  3.77669573e-01  9.91746485e-01 -1.73808947e-01 -3.77430737e-01
  1.41706973e-01 -6.00878775e-01  4.41664666e-01 -4.05263603e-01
  8.51022303e-02  1.85672700e-01 -6.47467747e-03  6.92536175e-01
 -4.91253793e-01  3.78734589e-01 -1.32684276e-01 -5.23463905e-01
 -3.82226110e-01  6.65980503e-02  3.20680201e-01  6.02832079e-01
 -3.69184732e-01 -3.52498382e-01  1.55600488e-01 -3.70579422e-01
 -2.75490105e-01  1.38546124e-01  4.38380092e-01 -5.66338956e-01
  2.13741526e-01 -4.58519042e-01  4.22842830e-01  3.83795619e-01
  9.62166041e-02 -7.06756711e-01 -1.99657440e-01  2.57501811e-01
  3.74362022e-01  3.30388173e-02 -2.46710181e-01 -2.37757638e-01
 -8.44248533e-01 -1.47765234e-01  2.80101031e-01  6.73321784e-01
  6.26941398e-02  5.73943436e-01 -5.69737852e-01  6.82345450e-01
 -7.87859336e-02  5.36466

In [ ]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()